prompt for task 1: Task: Generate a minimal Flask REST API with one route:
GET /hello → returns {"message": "Hello, AI Coding!"}. Include comments and a small runnable if __name__ == "__main__" block so it can be started with python task1_flask_app.py.

In [ ]:
# Minimal Flask app with one route: GET /hello
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def hello():
    """
    GET /hello
    Returns a JSON greeting.
    """
    return jsonify({"message": "Hello, AI Coding!"}), 200

if __name__ == "__main__":
    # Run the app on localhost:5000
    # Start with: python task1_flask_app.py
    app.run(host='0.0.0.0', port=5000, debug=True)


prompt for task 2: Task: Build a Student CRUD API using Flask with in-memory storage (list of dicts). Implement:

GET /students → list all students

POST /students → add a new student (expects JSON: {"name": "...", "age": 20})

PUT /students/<id> → update (name/age)

DELETE /students/<id> → delete
Return JSON responses and proper status codes; handle not-found and simple validation.

In [ ]:
# Flask Student API implementing CRUD operations using in-memory storage.
from flask import Flask, jsonify, request, abort

app = Flask(__name__)

# In-memory storage: list of students
# Each student is a dict: {"id": int, "name": str, "age": int}
students = []
_next_id = 1

def _get_next_id():
    global _next_id
    nid = _next_id
    _next_id += 1
    return nid

def find_student(sid):
    return next((s for s in students if s["id"] == sid), None)

@app.route('/students', methods=['GET'])
def list_students():
    """GET /students - return list of students"""
    return jsonify({"students": students}), 200

@app.route('/students', methods=['POST'])
def add_student():
    """POST /students - add a new student"""
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400
    payload = request.get_json()
    name = payload.get("name", "").strip()
    age = payload.get("age")
    if not name or not isinstance(age, int):
        return jsonify({"error": "Invalid payload. 'name' (non-empty) and 'age' (int) required."}), 400
    new_student = {"id": _get_next_id(), "name": name, "age": age}
    students.append(new_student)
    return jsonify(new_student), 201

@app.route('/students/<int:sid>', methods=['PUT'])
def update_student(sid):
    """PUT /students/<id> - update student"""
    student = find_student(sid)
    if student is None:
        return jsonify({"error": "Student not found"}), 404
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400
    payload = request.get_json()
    # Allow partial updates
    name = payload.get("name")
    age = payload.get("age")
    if name is not None:
        if not isinstance(name, str) or not name.strip():
            return jsonify({"error": "Invalid 'name'"}), 400
        student["name"] = name.strip()
    if age is not None:
        if not isinstance(age, int):
            return jsonify({"error": "Invalid 'age' (must be int)"}), 400
        student["age"] = age
    return jsonify(student), 200

@app.route('/students/<int:sid>', methods=['DELETE'])
def delete_student(sid):
    """DELETE /students/<id> - remove student"""
    student = find_student(sid)
    if student is None:
        return jsonify({"error": "Student not found"}), 404
    students.remove(student)
    return jsonify({"message": f"Student {sid} deleted"}), 200

if __name__ == "__main__":
    # Pre-populate with two sample students for testing convenience
    students.append({"id": _get_next_id(), "name": "Alice", "age": 21})
    students.append({"id": _get_next_id(), "name": "Bob", "age": 22})
    app.run(host='0.0.0.0', port=5001, debug=True)


prompt for task 3: Task: Add a search endpoint to the Student API that accepts query parameters and returns matching students. Example:

GET /students/search?name=ali → return students whose name contains "ali" (case-insensitive)

Also support min_age and max_age as optional numeric filters: /students/search?name=ali&min_age=20&max_age=25
Return JSON list of matched students and handle invalid query values gracefully.

In [ ]:
# Extends the students API with a search endpoint supporting query parameters.
from flask import Flask, jsonify, request

app = Flask(__name__)

# Example in-memory data - replace or import this from persistent student module if desired
students = [
    {"id": 1, "name": "Alice", "age": 21},
    {"id": 2, "name": "Bob", "age": 22},
    {"id": 3, "name": "Charlie", "age": 20},
    {"id": 4, "name": "Alisha", "age": 24},
]

@app.route('/students', methods=['GET'])
def list_students():
    return jsonify({"students": students}), 200

@app.route('/students/search', methods=['GET'])
def search_students():
    """
    GET /students/search?name=<term>&min_age=<n>&max_age=<n>
    - name: substring match (case-insensitive)
    - min_age, max_age: numeric filters
    """
    qname = request.args.get('name', '').strip()
    min_age = request.args.get('min_age')
    max_age = request.args.get('max_age')

    # Parse min_age and max_age if provided
    try:
        min_age_val = int(min_age) if (min_age is not None and min_age != '') else None
    except ValueError:
        return jsonify({"error": "min_age must be an integer"}), 400
    try:
        max_age_val = int(max_age) if (max_age is not None and max_age != '') else None
    except ValueError:
        return jsonify({"error": "max_age must be an integer"}), 400

    # Filter students
    def matches(s):
        if qname:
            if qname.lower() not in s['name'].lower():
                return False
        if (min_age_val is not None) and (s['age'] < min_age_val):
            return False
        if (max_age_val is not None) and (s['age'] > max_age_val):
            return False
        return True

    results = [s for s in students if matches(s)]
    return jsonify({"query": {"name": qname, "min_age": min_age_val, "max_age": max_age_val}, "results": results}), 200

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5002, debug=True)


prompt for task 4: Task: Create test scripts using Python's requests module that call the endpoints from Tasks 1–3. Include examples for:

GET /hello

CRUD flow for /students: POST new student, GET list, PUT update, DELETE student

GET /students/search with query params
The script should print responses, status codes, and readable JSON. Note: tests assume the respective servers are running on ports 5000 (task1), 5001 (task2), and 5002 (task3). Include error handling.

---



In [ ]:
# Test script using requests to exercise the APIs from Task 1-3.
import requests
import json
from pprint import pprint

def call_get(url):
    try:
        r = requests.get(url, timeout=5)
        print(f"GET {url} -> {r.status_code}")
        pprint(r.json())
    except Exception as e:
        print(f"GET {url} failed: {e}")

def call_post(url, payload):
    try:
        r = requests.post(url, json=payload, timeout=5)
        print(f"POST {url} -> {r.status_code}")
        try:
            pprint(r.json())
        except:
            print(r.text)
        return r
    except Exception as e:
        print(f"POST {url} failed: {e}")

def call_put(url, payload):
    try:
        r = requests.put(url, json=payload, timeout=5)
        print(f"PUT {url} -> {r.status_code}")
        try:
            pprint(r.json())
        except:
            print(r.text)
        return r
    except Exception as e:
        print(f"PUT {url} failed: {e}")

def call_delete(url):
    try:
        r = requests.delete(url, timeout=5)
        print(f"DELETE {url} -> {r.status_code}")
        try:
            pprint(r.json())
        except:
            print(r.text)
        return r
    except Exception as e:
        print(f"DELETE {url} failed: {e}")

if __name__ == "__main__":
    # Task 1: GET /hello (port 5000)
    print("\n--- Task 1: /hello ---")
    call_get("http://127.0.0.1:5000/hello")

    # Task 2: Student CRUD (port 5001)
    base2 = "http://127.0.0.1:5001/students"
    print("\n--- Task 2: Student CRUD ---")
    # Create a student
    resp = call_post(base2, {"name": "TestUser", "age": 19})
    if resp is None:
        print("POST failed; aborting Task 2 tests.")
    else:
        try:
            new = resp.json()
            new_id = new.get("id")
        except:
            new_id = None

        # List students
        call_get(base2)

        # Update the new student
        if new_id:
            call_put(f"{base2}/{new_id}", {"name": "TestUserUpdated", "age": 20})

            # Delete the new student
            call_delete(f"{base2}/{new_id}")

            # Confirm deletion
            call_get(base2)

    # Task 3: Search (port 5002)
    print("\n--- Task 3: /students/search ---")
    # Example queries
    call_get("http://127.0.0.1:5002/students/search?name=ali")
    call_get("http://127.0.0.1:5002/students/search?min_age=21&max_age=23")
    call_get("http://127.0.0.1:5002/students/search?name=char&min_age=18")